In [ ]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import nltk
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
nltk.download('punkt')

In [35]:
df = pd.read_csv('googleplaystore_user_reviews.csv')

# Remove missing values and clean the text
df = df.dropna(subset=['Sentiment', 'Translated_Review'])
df = df[df['Sentiment'].isin(['Positive', 'Negative', 'Neutral'])]
df['Translated_Review'] = df['Translated_Review'].str.lower().str.replace('[^\w\s]', '', regex=True)
df['Translated_Review'] = df['Translated_Review'].apply(word_tokenize)

# Convert the text to numerical sequences using the Keras Tokenizer
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['Translated_Review'])
X = tokenizer.texts_to_sequences(df['Translated_Review'])
X = pad_sequences(X, maxlen=200)
y = np.array(df['Sentiment'])
Y = label_encoder.fit_transform(y)



In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [38]:
# Define the model architecture
model = Sequential()
model.add(Embedding(10000, 128, input_length=X.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])

y_train = label_encoder.fit_transform(y_train)

In [ ]:
# Train the model on the data
early_stop = EarlyStopping(monitor='val_loss', patience=3)
#model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stop])
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.2, callbacks=[early_stop])